# input data

In [1]:
import pandas as pd


In [2]:

baseline = pd.read_csv('sub43data/sub-43_ses-01_task-RLbaseline_run-1_space-fsLR_den-91k_bold_timeseries.tsv', delimiter='\t')
baseline.shape

(219, 998)

In [3]:
rs43_fname = 'sub-43_ses-01_task-rest_run-1_space-fsLR_den-91k_bold_timeseries.tsv'
rs = pd.read_csv('sub43data/' + rs43_fname, delimiter='\t')
rs.shape

(297, 998)

In [4]:
learning_43_fname = 'sub-43_ses-01_task-RLlearning_run-1_space-fsLR_den-91k_bold_timeseries.tsv'
lrn = pd.read_csv('sub43data/' + learning_43_fname, delimiter='\t')
lrn.shape

(609, 998)

In [5]:
rs.columns

Index(['7Networks_LH_Vis_1', '7Networks_LH_Vis_2', '7Networks_LH_Vis_3',
       '7Networks_LH_Vis_4', '7Networks_LH_Vis_5', '7Networks_LH_Vis_6',
       '7Networks_LH_Vis_7', '7Networks_LH_Vis_8', '7Networks_LH_Vis_9',
       '7Networks_LH_Vis_10',
       ...
       '7Networks_RH_Default_pCunPCC_11', '7Networks_RH_Default_pCunPCC_12',
       '7Networks_RH_Default_pCunPCC_13', '7Networks_RH_Default_pCunPCC_14',
       '7Networks_RH_Default_pCunPCC_15', '7Networks_RH_Default_pCunPCC_16',
       '7Networks_RH_Default_pCunPCC_17', '7Networks_RH_Default_pCunPCC_18',
       '7Networks_RH_Cont_pCun_2', '7Networks_RH_Cont_pCun_4'],
      dtype='object', length=998)

# atlas

In [6]:
import nibabel as nib
import numpy as np

atlas_lh = nib.freesurfer.read_annot('atlas/Schaefer2018_1000Parcels_7Networks/lh.Schaefer2018_1000Parcels_7Networks_order.annot')
surf_labels_lh = atlas_lh[0]
atlas_rh = nib.freesurfer.read_annot('atlas/Schaefer2018_1000Parcels_7Networks/rh.Schaefer2018_1000Parcels_7Networks_order.annot')
surf_labels_rh = atlas_rh[0]

In [7]:
surf_labels_rh[surf_labels_rh != 0] += 500  # different labels for lh and rh
surf_labels = np.concatenate([surf_labels_lh, surf_labels_rh])

In [8]:
labels_lh = [x.decode() for x in atlas_lh[2]]
labels_rh = [x.decode() for x in atlas_rh[2]]
labels_rh.remove('Background+FreeSurfer_Defined_Medial_Wall')
regions = labels_lh + labels_rh

In [9]:
atlas_lh[1].shape

(501, 5)

In [10]:
REMOVED_REGIONS = set(regions) - set(rs.columns.tolist())
REMOVED_REGIONS

{'7Networks_RH_Cont_Cing_1',
 '7Networks_RH_Vis_33',
 'Background+FreeSurfer_Defined_Medial_Wall'}

## labels to plot surf 

it's also the mapping from 20484 2mm voxels to 1001 regions on brain surface

In [11]:
surf_labels

array([145, 214, 128, ..., 833, 833, 833], dtype=int32)

In [12]:
(surf_labels == 0).sum()

1743

In [13]:
(surf_labels == 1000).sum()

17

https://github.com/ThomasYeoLab/CBIG/blob/master/stable_projects/brain_parcellation/Schaefer2018_LocalGlobal/Parcellations/MNI/Centroid_coordinates/Schaefer2018_1000Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv

## removing regions

In [14]:
removed_labels = [regions.index(r) for r in REMOVED_REGIONS]
removed_labels
# it's the plus one

[0, 903, 533]

In [15]:
for r in REMOVED_REGIONS:
    regions.remove(r)

len(regions)

998

## corr mat

In [16]:
from nilearn.connectome import ConnectivityMeasure

In [17]:
# resting state data
correlation_measure = ConnectivityMeasure(kind='correlation')
corr_mat = correlation_measure.fit_transform([rs.values])[0]

In [18]:
# baseline 
correlation_measure_baseline = ConnectivityMeasure(kind='correlation')
corr_mat_baseline = correlation_measure_baseline.fit_transform([baseline.values])[0]

In [19]:
# learning
correlation_measure_lrn = ConnectivityMeasure(kind='correlation')
corr_mat_lrn = correlation_measure_lrn.fit_transform([lrn.values])[0]

# conn matrix

In [20]:
from nilearn import plotting

/Users/qasem/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


## reduced mat based on std

In [21]:
# Reduce matrix size, only for visualization purposes
mat_mask = np.where(np.std(corr_mat, axis=1) > 0.2)[0]
corr_mat_masked = corr_mat[mat_mask][:, mat_mask]
# Create corresponding region names
regions_list = ['%s_%s' % (h, r) for h in ['L', 'R'] for r in regions]
masked_regions = [regions_list[i] for i in mat_mask]

In [22]:
len(mat_mask)

636

In [23]:
mat_mask[:10] # e.g. no 5 and 6 

array([ 1,  2,  3,  4,  7,  8,  9, 10, 11, 12])

In [24]:
from matplotlib import pyplot as plt

# fig = plt.figure(figsize=(15,15))
# ax = plotting.plot_matrix(corr_mat_masked, labels=masked_regions,
#                                  vmax=0.8, vmin=-0.8, reorder=True,
#                                  figure=fig, # figure=(15, 15), 
#                                  )

# fig.savefig('con-mat.png')

## reordering

In [28]:
# reordered_labels = [lbl.get_text() for lbl in ax.axes.get_xticklabels()]
# len(reordered_labels)
# regions[:5], reordered_labels[:5]
# str_regions = [str(r) for r in regions]
# regions_ordering = [str_regions.index(r) for r in reordered_labels]
# corr_mat_reordered = corr_mat[regions_ordering][:, regions_ordering]
# plotting.plot_matrix(corr_mat_reordered, figure=(15, 15),
#                         labels=reordered_labels, reorder=False,
#                         vmax=0.8, vmin=-0.8)

https://github.com/nilearn/nilearn/issues/1633

## baseline and learning respecting to rs 

now that we have the orders, we can input that to the next cond_data

In [29]:
# corr_mat_baseline.shape
# corr_mat_baseline_reordered = corr_mat_baseline[regions_ordering][:, regions_ordering]

---

# plot surf

In [30]:
removed_labels

[0, 903, 533]

In [31]:
surf_labels

array([145, 214, 128, ..., 833, 833, 833], dtype=int32)

In [32]:
# Build Destrieux parcellation and mask
mask_removed = ~np.isin(surf_labels, removed_labels)

In [33]:
len(mask_removed), (mask_removed == False).sum()
# removed voxels due to being in removed regions

(20484, 1761)

In [105]:
len(set(surf_labels)), len(set(surf_labels[mask_removed]))

(1001, 998)

# Gradients

separately fitted on lrn

In [130]:
from brainspace.gradient import GradientMaps
gm_rs = GradientMaps(n_components=10, random_state=0, alignment="procrustes",
                      approach="pca")

gm_rs.fit(corr_mat, sparsity=0.9)
# shape of gm.gradients_[:, 0] is 1000
# fit on rs
gm_rs.gradients_[:1]

array([[-0.81803871, -1.74470451,  3.12175069,  0.15464395,  0.98718177,
         0.70315699,  0.04863302,  0.41439915,  0.47973416, -0.15317973]])

In [131]:
# separately fitted on lrn

gm_lrn1 = GradientMaps(n_components=10, random_state=0, alignment="procrustes",
                      approach="pca")

gm_lrn1.fit(corr_mat_lrn,
    sparsity=0.9)

gm_lrn1.gradients_[:1]

array([[-1.71148523,  2.02162346,  0.86332861,  1.71478229, -0.59207486,
         0.18172818, -2.24997682, -0.19248734, -0.17877626,  0.32336313]])

In [133]:
gm_rs.gradients_.shape

(998, 10)

---
combined

In [136]:

gm_lrn = GradientMaps(n_components=10, random_state=0, alignment="procrustes",
                      approach="pca")

gm_lrn.fit([corr_mat, corr_mat_lrn],
    sparsity=0.9)

gm_lrn.gradients_[0][:1]

array([[-0.81803871, -1.74470451,  3.12175069,  0.15464395,  0.98718177,
         0.70315699,  0.04863302,  0.41439915,  0.47973416, -0.15317973]])

In [140]:
gm_lrn.aligned_[0][:1]

array([[-0.81803871, -1.74470451,  3.12175069,  0.15464395,  0.98718177,
         0.70315699,  0.04863302,  0.41439915,  0.47973416, -0.15317973]])

In [137]:
gm_lrn.gradients_[1][:1]

array([[-1.71148523,  2.02162346,  0.86332861,  1.71478229, -0.59207486,
         0.18172818, -2.24997682, -0.19248734, -0.17877626,  0.32336313]])

In [139]:
gm_lrn.aligned_[1][:1]

array([[ 3.08404319,  0.86743908,  0.29022823,  0.81314477, -0.71786866,
        -0.63248146,  0.57938731,  0.14443662,  2.00863262,  0.07597737]])

aligned is different for the second input in list!

---
fit on rs then on lrn

In [141]:
gmr = GradientMaps(n_components=10, random_state=0, approach="pca")
gmr.fit(corr_mat)

gml = GradientMaps(n_components=10, random_state=0, alignment="procrustes", approach="pca",)
gml.fit(corr_mat_lrn, reference=gmr.gradients_)

gml.gradients_[:1]

array([[-1.71148523,  2.02162346,  0.86332861,  1.71478229, -0.59207486,
         0.18172818, -2.24997682, -0.19248734, -0.17877626,  0.32336313]])

In [142]:
gml.aligned_[:1]

array([[ 3.08404319,  0.86743908,  0.29022823,  0.81314477, -0.71786866,
        -0.63248146,  0.57938731,  0.14443662,  2.00863262,  0.07597737]])

works the same!

---

In [143]:
gmp = GradientMaps(n_components=10, random_state=0, approach="pca", alignment="procrustes", )
gmp.fit([corr_mat, corr_mat_lrn], reference=gmr.gradients_)

GradientMaps(alignment='procrustes', approach='pca', random_state=0)

In [144]:
gmp.gradients_[1][:1]

array([[-1.71148523,  2.02162346,  0.86332861,  1.71478229, -0.59207486,
         0.18172818, -2.24997682, -0.19248734, -0.17877626,  0.32336313]])

In [145]:
gmp.aligned_[1][:1]

array([[ 3.08404319,  0.86743908,  0.29022823,  0.81314477, -0.71786866,
        -0.63248146,  0.57938731,  0.14443662,  2.00863262,  0.07597737]])

In [146]:
gmp.aligned_[1].shape

(998, 10)

same again

---

In [147]:
from brainspace.utils.parcellation import map_to_labels

In [ ]:
# learning grads:

In [148]:
grad = map_to_labels(gml.gradients_[:, 0], surf_labels, mask=mask_removed, fill=np.nan)
grad2 = map_to_labels(gml.gradients_[:, 1], surf_labels, mask=mask_removed, fill=np.nan)

In [150]:
# aligned

grad_aligned = map_to_labels(gml.aligned_[:, 0], surf_labels, mask=mask_removed, fill=np.nan)
grad2_aligned = map_to_labels(gml.aligned_[:, 1], surf_labels, mask=mask_removed, fill=np.nan)

In [151]:
from brainspace.datasets import load_fsa5

surf_lh, surf_rh = load_fsa5()

In [152]:
text = ['Schaefer\n1000', '1st grad', '1st grad aligned', '2nd grad', '2nd grad aligned']
grad_data = [surf_labels, grad, grad_aligned, grad2, grad2_aligned]
color_map = ['tab20', 'viridis_r', 'viridis_r', 'viridis_r', 'viridis_r']

In [158]:
from brainspace.plotting import plot_hemispheres

plot_hemispheres(surf_lh=surf_lh, surf_rh=surf_rh,
array_name=grad_data, 
filename='subject 43 - baseline gradients',
    size=(1200, 1500), color_bar=True, 
    cmap=color_map,
    zoom=1.5, label_text=text)
